Loads and Validates Data

In [ ]:
# Neural Networks

# Class Notes:

# predict total number of bike rentals that occured?
# y is casual and registered combined.
# how do we want the data fed into the network?
# how will we use the data from 2020, the lockdown?
# combine values from hum and temp_c, feature enginere road conditions.
# trend from weather and covid.
# What were the exact dates the DC lockdown?
# frequency, average trend of busier times
# look at the tourist Season
# use both temp_c and temp_f

In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt

bikes = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/bikes.csv')
mini_holdout = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/biking_holdout_test_mini.csv')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
def get_day_of_year(row):
  date_list = row["dteday"].split("/")

  month = int(date_list[0])
  day = int(date_list[1])
  year = int(date_list[2]) # check for leap years

  if month == 1:
    days = day
  elif month == 2:
    days = day + 31
  elif month == 3:
    days = day + 31 + 28
  elif month == 4:
    days = day + 31 + 28 + 31
  elif month == 5:
    days = day + 31 + 28 + 31 + 30
  elif month == 6:
    days = day + 31 + 28 + 31 + 30 + 31
  elif month == 7:
    days = day + 31 + 28 + 31 + 30 + 31 + 30
  elif month == 8:
    days = day + 31 + 28 + 31 + 30 + 31 + 30 + 31
  elif month == 9:
    days = day + 31 + 28 + 31 + 30 + 31 + 30 + 31 + 31
  elif month == 10:
    days = day + 31 + 28 + 31 + 30 + 31 + 30 + 31 + 31 + 30
  elif month == 11:
    days = day + 31 + 28 + 31 + 30 + 31 + 30 + 31 + 31 + 30 + 31
  elif month == 12:
    days = day + 31 + 28 + 31 + 30 + 31 + 30 + 31 + 31 + 30 + 31 + 30

  if year % 4 == 0:
    days += 1

  return int(days)

def get_day_of_four_years(row):
  # there are 1461 days in four years but I want everything to be centered around election year
  year = int(row["dteday"].split("/")[2])
  day = int(row["dteday"].split("/")[1])

  days = ((year % 4) * 365) + day
  if year % 4 != 0:
    days += 1

  return int(days)

In [ ]:
def preprocess_bike_data(df):
    df["day_of_year"] = df.apply(get_day_of_year, axis=1)
    df["day_of_four_years"] = df.apply(get_day_of_four_years, axis=1)

    # Convert 'dteday' to datetime
    df['dteday'] = pd.to_datetime(df['dteday'], format='%m/%d/%Y')

    # Ensure 'hr' is an integer
    df['hr'] = df['hr'].astype(int)

    # Combine 'dteday' and 'hr' into a single 'datetime' column
    df['dteday'] = df.apply(lambda row: pd.Timestamp(row['dteday']) + pd.Timedelta(hours=row['hr']), axis=1)

    # Create year, month, and day columns
    df['year']        = df['dteday'].dt.year
    df['month']       = df['dteday'].dt.month
    df['day']         = df['dteday'].dt.day
    df['day_of_week'] = df['dteday'].dt.dayofweek

    return df


In [ ]:
bikes = preprocess_bike_data(bikes)

bikes.head()

In [ ]:
no_covid = bikes
from_ts = pd.to_datetime('2020-03-01 00:00:00')
to_ts = pd.to_datetime('2021-03-31 00:00:00')
no_covid = no_covid[(no_covid['dteday'] < from_ts) | (no_covid['dteday'] > to_ts)]


In [ ]:
'''
This is a chronological train/ test split.
'''

from sklearn.model_selection import train_test_split
from sklearn import preprocessing

# Get dummies
#bikes_dum = pd.get_dummies(bikes, columns=['hr', 'weathersit', 'season', 'holiday', 'workingday', 'year', 'month', 'day'])
bikes_dum = bikes

# Ensure the data is sorted by 'datetime' to maintain chronological order
bikes_dum = bikes_dum.sort_values(by='dteday')

# Create the target variable
bikes_dum['combined_y'] = bikes_dum['casual'] + bikes_dum['registered']
y = bikes_dum['combined_y']

# Define indices for chronological splitting
split_index = int(len(bikes_dum) * 0.9)

# Split the data chronologically
# Get the size of bikes for train/validation/testing
train_size = int(0.9 * len(bikes))
val_size = int(0.05 * len(bikes))
test_size = len(bikes) - train_size - val_size

# Split the data for train/val/test
train_data = bikes_dum[:train_size]
val_data = bikes_dum[train_size:train_size + val_size]
test_data = bikes_dum[train_size + val_size:]

# Separate features and target for training and testing sets
y_train = train_data['combined_y']
y_test = test_data['combined_y']
y_val = val_data['combined_y']

X_train = train_data.drop(['combined_y', 'dteday', 'casual', 'registered'], axis=1)
X_test = test_data.drop(['combined_y', 'dteday', 'casual', 'registered'], axis=1)
X_val = val_data.drop(['combined_y', 'dteday', 'casual', 'registered'], axis=1)

# Scale the data
minmax_scaler = preprocessing.MinMaxScaler()
X_train = minmax_scaler.fit_transform(X_train)
X_test = minmax_scaler.transform(X_test)

# Display the shapes of the datasets to verify
print(f'Training data shape: {X_train.shape}, Training target shape: {y_train.shape}')
print(f'Testing data shape: {X_test.shape}, Testing target shape: {y_test.shape}')
print(f'Validation data shape: {X_val.shape}, Validation target shape: {y_val.shape}')

In [ ]:
'''
no_covid = no_covid.sort_values(by='dteday')

# Create the target variable
no_covid['combined_y'] = no_covid['casual'] + no_covid['registered']
y = no_covid['combined_y']

# Define indices for chronological splitting
split_index = int(len(no_covid) * 0.9)

# Split the data chronologically
# Get the size of bikes for train/validation/testing
train_size2 = int(0.9 * len(bikes))
val_size = int(0.05 * len(bikes))
test_size = len(bikes) - train_size - val_size

# Split the data for train/val/test
train_data2 = no_covid[:train_size]
val_data2 = no_covid[train_size:train_size + val_size]
test_data2 = no_covid[train_size + val_size:]

# Separate features and target for training and testing sets
y_train2 = train_data2['combined_y']
y_test2 = test_data2['combined_y']
y_val2 = val_data2['combined_y']

X_train2 = train_data2.drop(['combined_y', 'dteday', 'casual', 'registered'], axis=1)
X_test2 = test_data2.drop(['combined_y', 'dteday', 'casual', 'registered'], axis=1)
X_val2 = val_data2.drop(['combined_y', 'dteday', 'casual', 'registered'], axis=1)

# Scale the data
minmax_scaler2 = preprocessing.MinMaxScaler()
X_train2 = minmax_scaler2.fit_transform(X_train)
X_test2 = minmax_scaler2.transform(X_test)

# Display the shapes of the datasets to verify
print(f'Training data shape: {X_train2.shape}, Training target shape: {y_train2.shape}')
print(f'Testing data shape: {X_test2.shape}, Testing target shape: {y_test2.shape}')
print(f'Validation data shape: {X_val2.shape}, Validation target shape: {y_val2.shape}')
'''

**Load Models**

In [ ]:
!pwd

In [ ]:
from google.colab import drive
import os
from keras.models import load_model
import tensorflow as tf

drive.mount('/content/drive')

In [ ]:
# I'm getting an error, I'm trying again with a .h5 file
top_file = "/content/drive/My Drive/Machine Learning Group 5/Bikes (Neural Nets)/"
sigmoid = tf.keras.models.load_model(top_file + 'sigmoid.h5')
tanh = tf.keras.models.load_model(top_file + 'tanh.h5')
# relu = tf.keras.models.load_model(top_file + 'my_model.keras')
# sigmoid_no_covid = tf.keras.models.load_model(top_file + 'sigmoid_no_covid.h5')
# tanh_no_covid = tf.keras.models.load_model(top_file + 'tanh_no_covid.h5')

Test Predictions

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, median_absolute_error
import numpy as np

In [ ]:
def get_predict(model):
  predictions = np.round(model.predict(X_test),1)
  return predictions

def get_r(predictions, y_test):
  r2 = r2_score(y_test,predictions)
  return r2

def get_mse(predictions, y_test):
  MSE = mean_squared_error(y_test,predictions)
  return MSE

def get_rmse(predictions, y_test):
  rmse = np.sqrt(get_mse(predictions, y_test))
  return rmse

def get_mean_ae(predictions, y_test):
  return mean_absolute_error(y_test,predictions)

def get_median_ae(predictions, y_test):
  return median_absolute_error(y_test,predictions)

In [ ]:
def get_scores(predictions, y_test):
  print("R2: ", get_r(predictions, y_test))
  print("MSE: ", get_mse(predictions, y_test))
  print("RMSE: ", get_rmse(predictions, y_test))
  print("Mean Absolute: ", get_mean_ae(predictions, y_test))
  print("Median Absolute: ", get_median_ae(predictions,y_test))


Hold Out Dataset

In [ ]:
# Load and preprocess mini_holdout
# mini_holdout = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/biking_holdout_test_mini.csv')
mini_holdout = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/bikes_december.csv')
mini_holdout = preprocess_bike_data(mini_holdout)

In [ ]:
# Ensure that mini_holdout has the same columns as the training data
missing_cols = set(bikes.columns) - set(mini_holdout.columns)
for col in missing_cols:
    mini_holdout[col] = 0

# Drop unnecessary columns in mini_holdout
mini_holdout = mini_holdout.drop(['dteday', 'datetime'], axis=1, errors='ignore')

# Align columns to match the training data
mini_holdout = mini_holdout[bikes.columns.drop(['dteday', 'casual', 'registered'])]

# Apply the same scaler fitted on the training data
mini_holdout = minmax_scaler.transform(mini_holdout)

# Check the shape of mini_holdout
print(mini_holdout.shape)

# Make predictions
mini_holdout_predictions = sigmoid.predict(mini_holdout)

# Format and export data
predictions_table = pd.DataFrame(mini_holdout_predictions, columns=['predictions'])
predictions_table.to_csv('team5-module4-predictions.csv', index=False)
print(predictions_table.value_counts())